# Segmenting and Clustering Neighborhoods in Toronto
For this assignment, you will be required to explore and cluster the neighborhoods in Toronto.

Start by creating a new Notebook for this assignment.
Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe like the one shown below:


In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
from sklearn.datasets.samples_generator import make_blobs

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
from bs4 import BeautifulSoup
import lxml
print('Libraries imported.')
print('Libraries imported.')

Libraries imported.
Libraries imported.


In [15]:
# download data and parse it (https://beautiful-soup-4.readthedocs.io/en/latest/)
request = requests.get("https://en.wikipedia.org/wiki/Provincial_city_(Vietnam)")
soup = BeautifulSoup(request.text, 'html.parser')
table=soup.find('table', attrs={'class':'wikitable sortable'})
#get header of dataframe from html text
headers=table.findAll('th')
for i, head in enumerate(headers):
    headers[i]=str(headers[i]).replace("<th>","").replace("</th>","").replace("\n","")
rows = table.findAll('tr')
rows=rows[1:len(rows)]
rows[1]

<tr>
<td><a class="mw-redirect" href="/wiki/B%E1%BA%A1c_Li%C3%AAu" title="Bạc Liêu">Bạc Liêu</a></td>
<td><a href="/wiki/B%E1%BA%A1c_Li%C3%AAu_Province" title="Bạc Liêu Province">Bạc Liêu</a></td>
<td>175.38</td>
<td>188,863</td>
<td>1,077</td>
<td>2010-08-27</td>
<td>2
</td></tr>

In [14]:
headers

['Name',
 'Province',
 'Area (km²)',
 'Population (person)',
 'Population density',
 'Year to become city',
 'Class']

In [17]:
# skip all meta symbols and line feeds between rows:
for i, row in enumerate(rows): 
    rows[i] = str(rows[i]).replace("</td></tr>","").replace("<tr>\n<td>","")
rows[1]

'<a class="mw-redirect" href="/wiki/B%E1%BA%A1c_Li%C3%AAu" title="Bạc Liêu">Bạc Liêu</a></td>\n<td><a href="/wiki/B%E1%BA%A1c_Li%C3%AAu_Province" title="Bạc Liêu Province">Bạc Liêu</a></td>\n<td>175.38</td>\n<td>188,863</td>\n<td>1,077</td>\n<td>2010-08-27</td>\n<td>2\n'

In [18]:
df=pd.DataFrame(rows)
df[headers] = df[0].str.split("</td>\n<td>", n = 6, expand = True) 

In [19]:
df

,0,Name,Province,Area (km²),Population (person),Population density,Year to become city,Class
0,"<a href=""/wiki/B%C3%A0_R%E1%BB%8Ba"" title=""Bà ...","<a href=""/wiki/B%C3%A0_R%E1%BB%8Ba"" title=""Bà ...","<a class=""mw-redirect"" href=""/wiki/B%C3%A0_R%E...",91.47,"122,424","1,338",2012-08-22,2\n
1,"<a class=""mw-redirect"" href=""/wiki/B%E1%BA%A1c...","<a class=""mw-redirect"" href=""/wiki/B%E1%BA%A1c...","<a href=""/wiki/B%E1%BA%A1c_Li%C3%AAu_Province""...",175.38,"188,863","1,077",2010-08-27,2\n
2,"<a href=""/wiki/B%E1%BA%A3o_L%E1%BB%99c"" title=...","<a href=""/wiki/B%E1%BA%A3o_L%E1%BB%99c"" title=...","<a href=""/wiki/L%C3%A2m_%C4%90%E1%BB%93ng_Prov...",232.56,"153,362",659,2010-04-08,"3<sup class=""reference"" id=""cite_ref-4""><a hre..."
3,"<a href=""/wiki/B%E1%BA%AFc_Giang"" title=""Bắc G...","<a href=""/wiki/B%E1%BA%AFc_Giang"" title=""Bắc G...","<a href=""/wiki/B%E1%BA%AFc_Giang_Province"" tit...",66.77,"157,439","2,358",2005-06-07,2\n
4,"<a href=""/wiki/B%E1%BA%AFc_K%E1%BA%A1n"" title=...","<a href=""/wiki/B%E1%BA%AFc_K%E1%BA%A1n"" title=...","<a href=""/wiki/B%E1%BA%AFc_K%E1%BA%A1n_Provinc...",137,"56,800",415,2015-03-11,3\n
5,"<a href=""/wiki/B%E1%BA%AFc_Ninh"" title=""Bắc Ni...","<a href=""/wiki/B%E1%BA%AFc_Ninh"" title=""Bắc Ni...","<a href=""/wiki/B%E1%BA%AFc_Ninh_Province"" titl...",82.61,"272,634","3,300",2006-01-26,1\n
6,"<a href=""/wiki/Bi%C3%AAn_H%C3%B2a"" title=""Biên...","<a href=""/wiki/Bi%C3%AAn_H%C3%B2a"" title=""Biên...","<a href=""/wiki/%C4%90%E1%BB%93ng_Nai_Province""...",264.08,"784,398","2,970",1976-02-01,1\n
7,"<a href=""/wiki/B%E1%BA%BFn_Tre"" title=""Bến Tre...","<a href=""/wiki/B%E1%BA%BFn_Tre"" title=""Bến Tre...","<a href=""/wiki/B%E1%BA%BFn_Tre_Province"" title...",71.12,"150,530","2,117",2009-08-11,"2<sup class=""reference"" id=""cite_ref-5""><a hre..."
8,"<a href=""/wiki/Bu%C3%B4n_Ma_Thu%E1%BB%99t"" tit...","<a href=""/wiki/Bu%C3%B4n_Ma_Thu%E1%BB%99t"" tit...","<a href=""/wiki/%C4%90%E1%BA%AFk_L%E1%BA%AFk_Pr...",377.18,"326,135",865,1995-01-21,"1<sup class=""reference"" id=""cite_ref-6""><a hre..."
9,"<a href=""/wiki/C%E1%BA%A9m_Ph%E1%BA%A3"" title=...","<a href=""/wiki/C%E1%BA%A9m_Ph%E1%BA%A3"" title=...","<a href=""/wiki/Qu%E1%BA%A3ng_Ninh_Province"" ti...",486.45,"195,800",403,2012-02-21,"2<sup class=""reference"" id=""cite_ref-7""><a hre..."


In [9]:

df.drop(columns=[0],inplace=True)

# skip not assigned boroughs:
df = df.drop(df[(df.Borough == "Not assigned")].index)
# give "Not assigned" Neighborhoods same name as Borough:
df.Neighbourhood.replace("Not assigned", df.Borough, inplace=True)

In [10]:
# copy Borough value to Neighborhood if NaN:
df.Neighbourhood.fillna(df.Borough, inplace=True)
# drop duplicate rows:
df=df.drop_duplicates()

# extract titles from columns
df.update(
    df.Neighbourhood.loc[
        lambda x: x.str.contains('title')
    ].str.extract('title=\"([^\"]*)',expand=False))

df.update(
    df.Borough.loc[
        lambda x: x.str.contains('title')
    ].str.extract('title=\"([^\"]*)',expand=False))

# delete Toronto annotation from Neighbourhood:
df.update(
    df.Neighbourhood.loc[
        lambda x: x.str.contains('Toronto')
    ].str.replace(", Toronto",""))
df.update(
    df.Neighbourhood.loc[
        lambda x: x.str.contains('Toronto')
    ].str.replace("\(Toronto\)",""))

# combine multiple neighborhoods with the same post code
df2 = pd.DataFrame({'Postcode':df.Postcode.unique()})
df2['Borough']=pd.DataFrame(list(set(df['Borough'].loc[df['Postcode'] == x['Postcode']])) for i, x in df2.iterrows())
df2['Neighborhood']=pd.Series(list(set(df['Neighbourhood'].loc[df['Postcode'] == x['Postcode']])) for i, x in df2.iterrows())
df2['Neighborhood']=df2['Neighborhood'].apply(lambda x: ', '.join(x))
df2.dtypes

df2.head()

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park (Toronto),Queen's Park


In [32]:
#add Geo-spatial data
dfll= pd.read_csv("http://cocl.us/Geospatial_data")
dfll.rename(columns={'Postal Code':'Postcode'}, inplace=True)
dfll.set_index("Postcode")
df2.set_index("Postcode")
toronto_data=pd.merge(df2, dfll)
toronto_data.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront , Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park (Toronto),Queen's Park,43.662301,-79.389494


In [33]:
address = 'Toronto, ON, Canada'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto, ON, Canada are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto, ON, Canada are 43.653963, -79.387207.


In [34]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto